In [1]:
import sys
sys.path.append('..')
from ift6758.features import games_to_table
import pandas as pd

In [2]:
df = pd.DataFrame(games_to_table(list(range(2017, 2024))))
df

game 2019020001
Using cached data for game id:  2019020001
using cached data for game id:  2019020001.json
using cached data for game id:  2019020001.json
printing game : Game 2019020001 on 2019-10-02 between (10, 'TOR', 5) and (9, 'OTT', 3)
game 2019020002
Using cached data for game id:  2019020002
using cached data for game id:  2019020002.json
using cached data for game id:  2019020002.json
printing game : Game 2019020002 on 2019-10-02 between (19, 'STL', 2) and (15, 'WSH', 3)
game 2019020003
Using cached data for game id:  2019020003
using cached data for game id:  2019020003.json
using cached data for game id:  2019020003.json
printing game : Game 2019020003 on 2019-10-02 between (22, 'EDM', 3) and (23, 'VAN', 2)
game 2019020004
Using cached data for game id:  2019020004
using cached data for game id:  2019020004.json
using cached data for game id:  2019020004.json
printing game : Game 2019020004 on 2019-10-02 between (54, 'VGK', 4) and (28, 'SJS', 1)
game 2019020005
Using cached 

,eventId,timeInPeriod,periodDescriptor_number,homeTeamDefendingSide,xCoord,yCoord,eventOwnerTeamId,scoringPlayerId,goalieInNetId,shotType,...,gameDate,season,homeTeam_abbrev,homeTeam_score,awayTeam_abbrev,awayTeam_score,awayTeam_id,homeTeam_id,type,shootingPlayerId
0,10,00:25,1,right,85.0,-1.0,9,8480801.0,8475883.0,tip-in,...,2019-10-02,20192020,TOR,5,OTT,3,9,10,Goal,NaN
1,15,01:31,1,right,-32.0,-2.0,10,NaN,8467950.0,snap,...,2019-10-02,20192020,TOR,5,OTT,3,9,10,ShotOnGoal,8476853.0
2,20,03:23,1,right,63.0,-6.0,9,NaN,8475883.0,snap,...,2019-10-02,20192020,TOR,5,OTT,3,9,10,ShotOnGoal,8476331.0
3,21,03:56,1,right,-59.0,-20.0,10,NaN,8467950.0,wrist,...,2019-10-02,20192020,TOR,5,OTT,3,9,10,ShotOnGoal,8476853.0
4,24,04:47,1,right,-42.0,-29.0,10,NaN,8467950.0,slap,...,2019-10-02,20192020,TOR,5,OTT,3,9,10,ShotOnGoal,8475197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120903,490,19:57,3,left,71.0,-16.0,28,NaN,8478499.0,backhand,...,2021-05-08,20202021,SJS,4,ARI,5,53,28,ShotOnGoal,8479393.0
120904,498,00:33,4,right,-68.0,-14.0,28,NaN,8478499.0,wrist,...,2021-05-08,20202021,SJS,4,ARI,5,53,28,ShotOnGoal,8474578.0
120905,500,00:57,4,right,56.0,11.0,53,NaN,8482246.0,wrist,...,2021-05-08,20202021,SJS,4,ARI,5,53,28,ShotOnGoal,8478856.0
120906,751,01:46,4,right,73.0,6.0,53,NaN,8482246.0,wrist,...,2021-05-08,20202021,SJS,4,ARI,5,53,28,ShotOnGoal,8478856.0


In [4]:
def determine_home_team_defending_side(event_row):
    if event_row['zoneCode'] != 'O':
        print("input has to be offensive zone event")
        return None
    team_id = event_row["eventOwnerTeamId"]
    home_team_id = event_row["homeTeam_id"]
    x_coord = event_row["xCoord"]

    if x_coord > 0 and team_id == home_team_id:
        return "left"
    elif x_coord < 0 and team_id == home_team_id:
        return "right"
    elif x_coord > 0 and team_id != home_team_id:
        return "right"
    elif x_coord < 0 and team_id != home_team_id:
        return "left"

In [6]:
from tqdm import tqdm

defending_side_dict = {}
def populate_home_team_defending_side(game_id, period_number):
    # filter for offensive zone events in the period of the game
    game_period_df = df[(df['id'] == game_id) & (df['periodDescriptor_number'] == period_number) & (df['zoneCode'] == 'O')]

    if len(game_period_df) == 0: # overtime periods may have no offensive zone events
        print("no offensive zone event found in period")
        print("game_id: ", game_id, "period_number: ", period_number)
        return None
        
    event = game_period_df.iloc[0] # take the first offensive zone event in the period
    if not pd.notna(event["homeTeamDefendingSide"]):
        return None
    defending_side = determine_home_team_defending_side(event)
    # if pd.notna(event["homeTeamDefendingSide"]) and defending_side: # sanity check with existing homeDefendingSides
    #     print(event["homeTeamDefendingSide"])
    #     if event["homeTeamDefendingSide"] != defending_side:
    #         print("defending side mismatch", defending_side)
    #         print(event)
    #     assert(event["homeTeamDefendingSide"] == defending_side) 
    if defending_side: # do not overwrite if no defending side is found
        defending_side_dict[f"{game_id}_{period_number}"]= defending_side
    
for game_id in tqdm(df['id'].unique()):
    for period_number in df[df['id'] == game_id]['periodDescriptor_number'].unique():
        populate_home_team_defending_side(game_id, period_number)

defending_side_dict

 65%|██████▍   | 1261/1950 [00:22<00:12, 56.04it/s]

no offensive zone event found in period
game_id:  2020020175 period_number:  3


100%|██████████| 1950/1950 [00:34<00:00, 57.15it/s]


{'2019020001_1': 'right',
 '2019020001_2': 'left',
 '2019020001_3': 'right',
 '2019020002_1': 'left',
 '2019020002_2': 'right',
 '2019020002_3': 'left',
 '2019020002_4': 'right',
 '2019020003_1': 'left',
 '2019020003_2': 'right',
 '2019020003_3': 'left',
 '2019020004_1': 'right',
 '2019020004_2': 'left',
 '2019020004_3': 'right',
 '2019020005_1': 'left',
 '2019020005_2': 'right',
 '2019020005_3': 'left',
 '2019020006_1': 'right',
 '2019020006_2': 'left',
 '2019020006_3': 'right',
 '2019020007_1': 'left',
 '2019020007_2': 'right',
 '2019020007_3': 'left',
 '2019020008_1': 'right',
 '2019020008_2': 'left',
 '2019020008_3': 'right',
 '2019020008_4': 'left',
 '2019020008_5': 'right',
 '2019020009_1': 'left',
 '2019020009_2': 'right',
 '2019020009_3': 'left',
 '2019020010_1': 'right',
 '2019020010_2': 'left',
 '2019020010_3': 'right',
 '2019020011_1': 'right',
 '2019020011_2': 'left',
 '2019020011_3': 'right',
 '2019020012_1': 'left',
 '2019020012_2': 'right',
 '2019020012_3': 'left',
 '201

In [12]:
# Iterate through the df and update the homeTeamDefendingSide column
for index, row in df.iterrows():
    key = f"{row['id']}_{row['periodDescriptor_number']}"
    if key in defending_side_dict:
        df.at[index, 'homeTeamDefendingSide'] = defending_side_dict[key]

df.head()

,eventId,timeInPeriod,periodDescriptor_number,homeTeamDefendingSide,xCoord,yCoord,eventOwnerTeamId,shootingPlayerId,goalieInNetId,shotType,...,gameDate,season,homeTeam_abbrev,homeTeam_score,awayTeam_abbrev,awayTeam_score,awayTeam_id,homeTeam_id,type,scoringPlayerId
0,54,00:38,1,right,-36.0,-28.0,52,8477504.0,8475883.0,wrist,...,2017-10-04,20172018,WPG,2,TOR,7,10,52,ShotOnGoal,NaN
1,55,00:49,1,right,-75.0,1.0,52,8473574.0,8475883.0,wrist,...,2017-10-04,20172018,WPG,2,TOR,7,10,52,ShotOnGoal,NaN
2,58,01:03,1,right,-73.0,10.0,52,8473412.0,8475883.0,backhand,...,2017-10-04,20172018,WPG,2,TOR,7,10,52,ShotOnGoal,NaN
3,62,01:46,1,right,80.0,-3.0,10,8470611.0,8473461.0,wrist,...,2017-10-04,20172018,WPG,2,TOR,7,10,52,ShotOnGoal,NaN
4,68,03:42,1,right,-44.0,-21.0,52,8479339.0,8475883.0,snap,...,2017-10-04,20172018,WPG,2,TOR,7,10,52,ShotOnGoal,NaN


In [13]:
df[df.duplicated()]

,eventId,timeInPeriod,periodDescriptor_number,homeTeamDefendingSide,xCoord,yCoord,eventOwnerTeamId,shootingPlayerId,goalieInNetId,shotType,...,gameDate,season,homeTeam_abbrev,homeTeam_score,awayTeam_abbrev,awayTeam_score,awayTeam_id,homeTeam_id,type,scoringPlayerId


In [14]:
df.to_csv("../ift6758/data/preprocessed_data/preprocessed_game_data_team_side.csv", index=False)

In [ ]:
unique_event_types = df['type'].unique()
unique_event_types

In [ ]:
events_with_no_zone_code = df[df['zoneCode'].isna()]
events_with_no_zone_code

In [ ]:
events_with_no_shot_type = df[df['shotType'].isna()]
events_with_no_shot_type

In [ ]:
events_with_no_goalie = df[df['goalieInNetId'].isna()]
events_with_no_goalie # net empty of no goalie ?

In [ ]:
# errors ?
events_with_no_x_coord = df[df['xCoord'].isna()]
events_with_no_x_coord

In [ ]:
events_with_no_y_coord = df[df['yCoord'].isna()]
events_with_no_y_coord